In [1]:
# goals:
# run VQE on all backends
# run Quantum Volume simulator on all backends
# figure out entanglement scheme on all backends

# so how to transpile VQE scheme for backends?

In [3]:
import numpy as np
from qiskit import QuantumCircuit

def test_circuit():
    circuit = QuantumCircuit(4)

    circuit.h([0, 1, 2, 3])
    circuit.x([0, 1])
    circuit.y(2)
    circuit.z(3)
    circuit.s(0)
    circuit.sdg(1)
    circuit.t(2)
    circuit.tdg(3)
    circuit.rx(np.pi / 4, 0)
    circuit.ry(np.pi / 2, 1)
    circuit.rz(3 * np.pi / 4, 2)
    circuit.p(np.pi / 8, 3)
    circuit.sx(0)
    circuit.sxdg(1)
    circuit.iswap(2, 3)
    circuit.swap([0, 1], [2, 3])
    circuit.cx(0, 1)
    circuit.cp(np.pi / 4, 2, 3)

    return circuit
circuit = test_circuit()

# IBM Devices

In [4]:
# ibm backends:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeBrisbane, FakeOsaka, FakeKyoto, FakeSherbrooke

backend = AerSimulator.from_backend(FakeBrisbane())
# all these backends have 127 qubits.

In [5]:
# ibm transpiling code: (from VQE qiskit docs)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
circuit_isa = pm.run(circuit)

# from some https://github.com/Qiskit/qiskit/issues/9717
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService

# service = QiskitRuntimeService()
backend = FakeBrisbane() # or service.get_backend("ibm_lagos")
circuit = transpile(circuit, backend=backend)  # throws error

# circuit_isa.draw(output='mpl')
# circuit.draw(output='mpl')
":trol:"

':trol:'

# Braket Devices
Transpiling

In [7]:
import qbraid.runtime
from qbraid.programs import QPROGRAM_REGISTRY
from qbraid.interface import (
    circuits_allclose,
    assert_allclose_up_to_global_phase,
    random_circuit,
)
from qbraid.transpiler import transpile


QPROGRAM_REGISTRY

{'cirq': cirq.circuits.circuit.Circuit,
 'qiskit': qiskit.circuit.quantumcircuit.QuantumCircuit,
 'braket': braket.circuits.circuit.Circuit,
 'braket_ahs': braket.ahs.analog_hamiltonian_simulation.AnalogHamiltonianSimulation,
 'openqasm3': openqasm3.ast.Program,
 'qasm2': str,
 'qasm3': str,
 'ionq': ~IonQDict}

In [11]:
braket_circuit = transpile(circuit, "braket")
print(braket_circuit)

"""
You may need to `pip install qiskit-braket-provider`. However, from the docs:

While the qiskit-braket-provider package offers direct conversions, this comes with a trade-off.
It eliminates intermediate steps but tends to be more prone to errors, particularly with unsupported
gates, compared to the qBraid transpiler. To leverage the best of both, you can integrate the
qiskit-braket-provider's direct conversion into the qBraid transpiler's conversion graph.
Consequently, when converting from Braket to Qiskit, the transpiler will first try the direct
conversion through the qiskit-braket-provider. If this conversion fails, it will automatically
revert to one of the other native qBraid conversion paths
"""

T   : │      0      │     1      │      2      │      3      │   4   │      5      │      6      │   7   │      8      │      9      │     10      │     11      │ 12  │     13     │  14   │     15      │     16      │  17   │     18      │     19      │     20     │         21          │     22      │     23      │      24       │     25      │     26      │     27     │         28          │     29      │     30      │  31   │     32      │ 33  │     34     │  35   │     36      │     37      │  38   │ 39  │     40     │      41       │     42      │ 43  │     44     │ 45  │  46   │     47      │ 48  │     49     │ 50  │
GP : │      0      │     0      │      0      │      0      │   0   │      0      │      0      │   0   │      0      │      0      │      0      │      0      │  0  │     0      │   0   │      0      │      0      │   0   │      0      │      0      │     0      │      0      │   0   │      0      │      0      │   0   │   0   │      0      │      0      │     0     

"\nYou may need to `pip install qiskit-braket-provider`. However, from the docs:\n\nWhile the qiskit-braket-provider package offers direct conversions, this comes with a trade-off.\nIt eliminates intermediate steps but tends to be more prone to errors, particularly with unsupported\ngates, compared to the qBraid transpiler. To leverage the best of both, you can integrate the\nqiskit-braket-provider's direct conversion into the qBraid transpiler's conversion graph.\nConsequently, when converting from Braket to Qiskit, the transpiler will first try the direct\nconversion through the qiskit-braket-provider. If this conversion fails, it will automatically\nrevert to one of the other native qBraid conversion paths\n"

In [13]:
from qbraid.transpiler import transpile

qasm3_program = transpile(circuit, 'qasm3')

from qbraid.programs import load_program
qprogram = load_program(qasm3_program)

from qbraid.runtime.ionq import IonQProvider
provider = IonQProvider('')
device = provider.get_device("qpu.harmony")

print(qprogram.transform(device))
print(qprogram.program)

CompilationError: Rebasing the specified quantum program to the provided basis gate set {'rx', 't', 'cx', 'sdg', 'sx', 'ry', 'z', 'x', 'y', 'rz', 'h', 'tdg', 'sxdg', 'swap', 's'} is not supported.

In [ ]:
import qiskit.qasm3
circuit = qiskit.qasm3.loads(qprogram.program)
circuit.draw("mpl")